<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/ExothermicTubularReactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig

In [37]:
jnp.set_printoptions(precision=3, linewidth=200)

In [85]:
N=5
q=100. #L/min
ca_in = 1. #mol/L
T_in = 350. #K
Vtotal = 100. #L
V = Vtotal/N  #L
rho = 1000. #g/L
C = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750. #K
k0 = 7.2e10 #1/min

UAtot = 5e4 #J/(min K)
UA = UAtot/N

Tc_ss = 300. #K
Ca_ss_guess =  np.full(N, 0.5) #mol/L
T_ss_guess = np.full(N, 350.) #K
tend = 1000. #min

In [86]:
def rhs(t, vec):
    Tc=Tc_ss
    ca, T = jnp.split(vec,2)
    dca, dT = jnp.zeros_like(ca), jnp.zeros_like(T)
    k=k0*jnp.exp(-ER/T)
    dca=dca.at[0].set(q*(ca_in - ca[0])/V - k[0]*ca[0])
    dT=dT.at[0].set((rho*q*C*(T_in-T[0]) - UA*(T[0]-Tc) + negHr*V*k[0]*ca[0])/(rho*V*C))

    dca=dca.at[1:].set(q*(ca[:-1] - ca[1:])/V - k[1:]*ca[1:])
    dT=dT.at[1:].set((rho*q*C*(T[:-1]-T[1:]) - UA*(T[1:]-Tc) + negHr*V*k[1:]*ca[1:])/(rho*V*C))
    return jnp.r_[dca/ca_in, dT/Tc_ss]

In [87]:
rhs_jac = jax.jit(jax.jacobian(rhs, argnums=1))

In [88]:
vec0=sp.optimize.root(lambda v: rhs(0., v), np.r_[Ca_ss_guess, T_ss_guess], jac = lambda v: rhs_jac(0., v), method='lm',options=dict(ftol=1e-15) ).x

In [89]:
res=sp.integrate.solve_ivp(rhs, (0, tend), vec0, method='Radau', dense_output=True, jac=rhs_jac)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.273e-01 ...  8.531e+02  1.000e+03]
        y: [[ 9.007e-01  9.019e-01 ...  5.296e-03  5.296e-03]
            [ 8.010e-01  8.014e-01 ...  2.522e-04  2.522e-04]
            ...
            [ 3.466e+02  3.466e+02 ...  3.642e+02  3.642e+02]
            [ 3.413e+02  3.413e+02 ...  3.452e+02  3.452e+02]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7a2773dce480>
 t_events: None
 y_events: None
     nfev: 547
     njev: 29
      nlu: 114

In [90]:
tplot=np.linspace(0,tend,200)
cplot, Tplot = np.split(res.sol(tplot),2,axis=0)

In [91]:

fig = make_subplots(rows=1, cols=2)
for i, (c, T) in enumerate(zip(cplot,Tplot)):
    fig.add_scatter(x=tplot, y=c, row=1, col=1, name=f'c{i+1}')
    fig.add_scatter(x=tplot, y=T, row=1, col=2, name=f'T{i+1}')
fig.update_layout(height=600, width=1000)